In [1]:
import numpy as np
import matplotlib.pyplot as plt
import vtk
import os


In [2]:
def rdp(points, epsilon):
    """
    Simplify a curve using the Ramer-Douglas-Peucker algorithm.

    Parameters:
    - points: List of (x, y) coordinates representing the curve.
    - epsilon: The maximum distance allowed between the original curve and the simplified curve.

    Returns:
    - List of simplified (x, y) coordinates.
    """
    if len(points) <= 2:
        return points

    # Find the point with the maximum distance
    dmax = 0
    index = 0
    end = len(points) - 1
    for i in range(1, end):
        d = np.linalg.norm(np.cross(
            np.array(points[end]) - np.array(points[0]),
            np.array(points[i]) - np.array(points[0])
        )) / np.linalg.norm(np.array(points[end]) - np.array(points[0]))

        if d > dmax:
            index = i
            dmax = d

    # If max distance is greater than epsilon, recursively simplify
    if dmax > epsilon:
        rec_results1 = rdp(points[:index+1], epsilon)
        rec_results2 = rdp(points[index:], epsilon)

        # Combine the results
        result = rec_results1[:-1] + rec_results2
    else:
        result = [points[0], points[end]]

    return result


In [3]:
def interpolar (centerline, epsilon = 0.05):

    resampleada = vtk.vtkPolyData()
    cellarray = vtk.vtkCellArray()
    points = vtk.vtkPoints()
    lista_puntos = []
    lista_rama = []
    for j in range(centerline.GetNumberOfCells()):#iterar por ramas  
        numberOfCellPoints = centerline.GetCell(j).GetNumberOfPoints();# number of points of the branch
        for i in range(numberOfCellPoints):
            originalPointId = centerline.GetCell(j).GetPointId(i)
            point = centerline.GetPoints().GetPoint(originalPointId)
            lista_puntos.append(list(point))
            
        #Ya tengo la lista de puntos de la rama entonces la resampleo y la agrego a la linea punto y polydata
        rama_resampleada = rdp(lista_puntos, epsilon)
        polyline = vtk.vtkPolyLine()
        for point in rama_resampleada:
            newPointId = points.InsertNextPoint(point)
            polyline.GetPointIds().InsertNextId(newPointId)
        cellarray.InsertNextCell(polyline)
        lista_rama.append(lista_puntos)
        lista_puntos = []
    resampleada.SetPoints(points)
    resampleada.SetLines(cellarray)
    return resampleada

In [4]:
centerlines = os.listdir("centerlines")
reader = vtk.vtkXMLPolyDataReader()
writer = vtk.vtkXMLPolyDataWriter()
for file in centerlines:
    if file not in os.listdir("resample_Eps01/centerlines/") and file.split(".")[1] == "vtp":
        print("calculando: ", file)
        reader.SetFileName("centerlines/" + file)
        reader.Update()
        # Get the polydata
        centerline = reader.GetOutput()
        resampleada = interpolar(centerline, 0.1)
        writer.SetFileName("resample_Eps01/centerlines/" + file)
        writer.SetInputData(resampleada)
        writer.Write()

calculando:  ArteryObjAN1-0-network.vtp
calculando:  ArteryObjAN1-11-network.vtp
calculando:  ArteryObjAN1-12-network.vtp
calculando:  ArteryObjAN1-15-network.vtp
calculando:  ArteryObjAN1-16-network.vtp
calculando:  ArteryObjAN1-17-network.vtp
calculando:  ArteryObjAN1-19-network.vtp
calculando:  ArteryObjAN1-2-network.vtp
calculando:  ArteryObjAN1-4-network.vtp
calculando:  ArteryObjAN1-5-network.vtp
calculando:  ArteryObjAN1-6-network.vtp
calculando:  ArteryObjAN1-7-network.vtp
calculando:  ArteryObjAN11-0-network.vtp
calculando:  ArteryObjAN11-1-network.vtp
calculando:  ArteryObjAN11-11-network.vtp
calculando:  ArteryObjAN11-16-network.vtp
calculando:  ArteryObjAN11-17-network.vtp
calculando:  ArteryObjAN11-2-network.vtp
calculando:  ArteryObjAN11-3-network.vtp
calculando:  ArteryObjAN11-4-network.vtp
calculando:  ArteryObjAN11-5-network.vtp
calculando:  ArteryObjAN11-6-network.vtp
calculando:  ArteryObjAN11-8-network.vtp
calculando:  ArteryObjAN116-0-network.vtp
calculando:  Arter

: 